In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

In [3]:
store = FeatureStore(repo_path="../feature_pipeline/feature_store/feature_repo")

In [4]:
entity_sql = f"""
    SELECT
        parent_asin,
        timestamp as event_timestamp
    FROM {store.get_data_source("parent_asin_rating_stats_source").get_table_query_string()}
    WHERE timestamp BETWEEN '2019-01-01' and '2021-12-31'
"""
training_df = store.get_historical_features(
    entity_df=entity_sql,
    features=[
        "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d",
    ],
).to_df()

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/feast/infra/utils/postgres/connection_utils.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [5]:
training_df

,parent_asin,event_timestamp,parent_asin_rating_cnt_365d
0,B00ASLSQHK,2019-06-18 19:14:47.758,0
1,B003FVYXY0,2019-02-20 19:45:07.082,4
2,B00LT0ZWJE,2019-04-24 13:56:55.072,4
3,B00RXEWOAA,2019-08-30 21:06:05.171,6
4,B00HNFPZF0,2019-08-09 15:35:42.549,2
...,...,...,...
26249,B014UWIKX2,2021-03-22 23:43:29.604,1
26250,B01LW71IBJ,2020-02-29 02:11:23.770,5
26251,B01MS1PMML,2019-01-16 05:30:29.053,6
26252,B08F1P3BCC,2021-02-09 20:51:29.752,42


In [16]:
# get online features

features = store.get_online_features(
    features=[
        "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d",
    ],
    entity_rows=[
        {
            "parent_asin": "B00RXEWOAA",
        }
    ],
).to_dict()

In [17]:
features

{'parent_asin': ['B00RXEWOAA'], 'parent_asin_rating_cnt_365d': [2]}

In [31]:
# get online features

features = store.get_online_features(
    features=[
        "user_rating_stats_fresh:user_rating_list_10_recent_asin",
        "user_rating_stats_fresh:user_rating_list_10_recent_asin_timestamp",
        # "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d"
    ],
    entity_rows=[
        {
            "user_id": "AE22236AFRRSMQIKGG7TPTB75QEA",
            # "parent_asin": "B00RXEWOAA",
        }
    ],
).to_dict()

In [24]:
features

{'user_id': ['AE22236AFRRSMQIKGG7TPTB75QEA'],
 'user_rating_list_10_recent_asin_timestamp': ['1368320561,1426609328,1426609961,1530219104'],
 'user_rating_list_10_recent_asin': ['B008GFRE5A,B00M4I1BAY,B08QKWQS9J,B000WYVBR0']}

In [34]:
# Get from unified feature service

feature_service = store.get_feature_service("sequence_stats_v1")

features = store.get_online_features(
    features=feature_service, 
    entity_rows=[
        {
            "user_id": "AE22236AFRRSMQIKGG7TPTB75QEA",
            # "parent_asin": "B00RXEWOAA",
        }
    ],
)
features.to_dict()

{'user_id': ['AE22236AFRRSMQIKGG7TPTB75QEA'],
 'user_rating_list_10_recent_asin_timestamp': ['1368320561,1426609328,1426609961,1530219104'],
 'user_rating_list_10_recent_asin': ['B008GFRE5A,B00M4I1BAY,B08QKWQS9J,B000WYVBR0']}

# Server

This assumes that you've already run:

- feast serve

In [19]:
import httpx

from pydantic import BaseModel
from typing import Dict, List, Optional, Any
from datetime import datetime


In [20]:
class FeatureRequest(BaseModel):
    feature_service: str
    entities: Dict[str, List[str]]

In [21]:
request = FeatureRequest(
    feature_service="sequence_stats_v1",
    entities={
        "user_id": ["AE22236AFRRSMQIKGG7TPTB75QEA"],
    }
)

request.model_dump()

{'feature_service': 'sequence_stats_v1',
 'entities': {'user_id': ['AE22236AFRRSMQIKGG7TPTB75QEA']}}

In [22]:
with httpx.Client() as client:
    response = client.post(
        "http://138.2.61.6:6566/get-online-features",
        json=request.model_dump(),
    )
    response.raise_for_status()
    features = response.json()

In [23]:
features

{'metadata': {'feature_names': ['user_id',
   'user_rating_list_10_recent_asin_timestamp',
   'user_rating_list_10_recent_asin']},
 'results': [{'values': ['AE22236AFRRSMQIKGG7TPTB75QEA'],
   'statuses': ['PRESENT'],
   'event_timestamps': ['1970-01-01T00:00:00Z']},
  {'values': ['1368320561,1426609328,1426609961,1530219104'],
   'statuses': ['PRESENT'],
   'event_timestamps': ['2018-12-26T23:25:00.477Z']},
  {'values': ['B008GFRE5A,B00M4I1BAY,B08QKWQS9J,B000WYVBR0'],
   'statuses': ['PRESENT'],
   'event_timestamps': ['2018-12-26T23:25:00.477Z']}]}

In [24]:
class FeatureRequestResult(BaseModel):
    class Metadata(BaseModel):
        feature_names: List[str]

    class Result(BaseModel):
        values: List[Optional[str]]
        statuses: List[str]
        event_timestamps: List[datetime]

    metadata: Metadata
    results: List[Result]
    

    def get_feature(self, feature_name: str) -> Any:
        try:
            feature_idx = self.metadata.feature_names.index(feature_name)
        except ValueError:
            raise ValueError(f"Feature '{feature_name}' not found.")
        
        feature_results = self.results
        
        def get_feature_values(idx):
            return feature_results[idx].values[0]
        
        feature_value = get_feature_values(feature_idx)
        
        if "list" in feature_name and isinstance(feature_value, str):
            # If the feature is a list, we need to convert it to a Python list
            if feature_value is not None:
                feature_value = feature_value.split(",")
                
        return feature_value
        

In [25]:
FeatureRequestResult.model_validate(features).get_feature("user_id")

'AE22236AFRRSMQIKGG7TPTB75QEA'

In [26]:
FeatureRequestResult.model_validate(features).get_feature("user_rating_list_10_recent_asin")

['B008GFRE5A', 'B00M4I1BAY', 'B08QKWQS9J', 'B000WYVBR0']

In [27]:
FeatureRequestResult.model_validate(features)

FeatureRequestResult(metadata=Metadata(feature_names=['user_id', 'user_rating_list_10_recent_asin_timestamp', 'user_rating_list_10_recent_asin']), results=[Result(values=['AE22236AFRRSMQIKGG7TPTB75QEA'], statuses=['PRESENT'], event_timestamps=[datetime.datetime(1970, 1, 1, 0, 0, tzinfo=TzInfo(UTC))]), Result(values=['1368320561,1426609328,1426609961,1530219104'], statuses=['PRESENT'], event_timestamps=[datetime.datetime(2018, 12, 26, 23, 25, 0, 477000, tzinfo=TzInfo(UTC))]), Result(values=['B008GFRE5A,B00M4I1BAY,B08QKWQS9J,B000WYVBR0'], statuses=['PRESENT'], event_timestamps=[datetime.datetime(2018, 12, 26, 23, 25, 0, 477000, tzinfo=TzInfo(UTC))])])